In [ ]:
import os
import argparse
import aifc
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import csv
import librosa

In [ ]:


def read_aiff(path):
    with aifc.open(path, 'r') as f:
        audio = f.readframes(f.getnframes())
        signal = np.frombuffer(audio, dtype=np.int16).byteswap().astype(np.float32)
    signal -= np.mean(signal)
    signal /= (np.std(signal) + 1e-8)
    return signal

def compute_spectrogram(signal, sr, n_fft=512, n_mels=128):
    mel = librosa.feature.melspectrogram(y=signal, sr=sr, n_fft=n_fft, hop_length=128, n_mels=n_mels, fmin=20, fmax=sr // 2)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    return mel_db

def spectrogram_to_tensor(spec, size=(224, 224)):
    # Convert to 3-channel image
    img = Image.fromarray(spec).convert("RGB")
    img = img.resize(size, Image.BICUBIC)
    img_tensor = torch.from_numpy(np.array(img)).permute(2, 0, 1).float() / 255.0
    return img_tensor

def main():
    # Example usage (replace with your actual arguments)
    data_dir = "path/to/your/audio/files"  # Replace with your data directory
    label_csv = "path/to/your/labels.csv"  # Replace with your label CSV file
    processed_dir = "path/to/output/directory" # Replace with your output directory
    sampling_rate = 2000  # Replace with your sampling rate if different

    os.makedirs(processed_dir, exist_ok=True)

    with open(label_csv, 'r') as f:
        reader = csv.reader(f)
        rows = list(reader)[1:]  # Skip the header row

    for fname, label in tqdm(rows, desc="Processing audio files"):
        path = os.path.join(data_dir, fname)
        if not os.path.exists(path):
            print(f"Warning: File not found - {path}")
            continue

        try:
            signal = read_aiff(path)
            spec = compute_spectrogram(signal, sampling_rate)
            tensor = spectrogram_to_tensor(spec)

            # Save the tensor (example using torch.save)
            output_filename = os.path.splitext(fname)[0] + ".pt"  # Or another suitable extension
            output_path = os.path.join(processed_dir, output_filename)
            torch.save(tensor, output_path)

        except Exception as e:
            print(f"Error processing file {fname}: {e}")

if __name__ == "__main__":
    main()
